# Multimodal RAG

In my other notebook, [GPT Vision in RAG](./powerpoint.ipynb), we use vision to help enrich our content during ingestion, prior to chat. By using vision, we are able to create robust descriptions of our complex slides that contain charts and graphs, and bring some amazing value to our end users.

In this post, we're going to explore using 4o vision in our chat, that is, send images in with the users' questions, instead of relying on the vision descsription of that slide. 

## The R in multi-modal RAG

Retrieving the images is the most important piece: send GPT the right image for the question. This gets tricky when we're searching for images. If we were to embed the images, we could retrieve images using vectors. My experience, vector searching doesn't suffice for text search, a hybrid approach improves search relevancy. If I were to apply the same assumptions to embedding images, I think we'll still need some level of textual description to help find the most relevant images. Maybe I'm wrong. I won't be exploring multi-modal embeddings and retrieval here. I will in a later post.

Let's create our "database" of slides and search. I reran the previous notebook and took those values.

In [1]:
all_data = [
    {
        "slide_number": 1,
        "text": [
            "My Comic Book Collection",
            "As of Dec 14, 2024"
        ],
        "description": "The image is a collage of comic book covers with a central text box overlay. Each comic book cover is distinct, displaying various characters and art styles. Here's a detailed description:\n\n1. **Central Text Box**: A large, dark gray rectangle is centered in the image, with the text \"My Comic Book Collection\" in bold white lettering. Below this, it says \"As of Dec 14, 2024\" in smaller white text.\n\n2. **Comic Covers**:\n   - **Top Row**:\n     - *Left*: A cover of \"The Amazing Spider-Man\" featuring Spider-Man in his iconic red and blue suit, swinging on a web.\n     - *Middle Left*: \"Batman 89\" shows Batman standing in front of a dark, urban backdrop with a full moon.\n     - *Middle Right*: \"Nebula\" depicts a futuristic character with mechanical elements and vibrant colors.\n     - *Right*: \"Eve\" shows a girl with eyes closed, inside a circular window, hinting at a science fiction theme.\n\n   - **Bottom Row**:\n     - *Left*: \"Scarlet Witch\" has a striking image of a woman in red, with intense green eyes.\n     - *Middle Left*: A comic with a woman in a pirate-like outfit, holding a sword.\n     - *Middle Right*: A vibrant and colorful comic cover with a group of characters in action poses.\n     - *Right*: A retro-style comic featuring \"The Hulk\" with a dynamic and bold illustration.\n\n3. **Arrangement**: The comic covers are arranged in two rows of four, framing the central text box. Each cover has vivid, engaging artwork, highlighting the variety and diversity in the comic collection.\n\nOverall, the image is a visually engaging showcase of the individual's diverse comic book collection as of December 14, 2024."
    },
    {
        "slide_number": 2,
        "text": [
            "Comic Books",
            "Comic books started in 1938 with the introduction of Action Comics #1 marking the debut of Superman. Over the years, comic books have sold for over $3.5M, making comic books investment-grade collectibles."
        ],
        "description": "The image is a slide titled \"Comic Books.\" The title appears at the top in large, bold black text. Below the title is a paragraph in smaller black text. It reads: \"Comic books started in 1938 with the introduction of Action Comics #1 marking the debut of Superman. Over the years, comic books have sold for over $3.5M, making comic books investment-grade collectibles.\"\n\nTo the right of the text is an image of the cover of \"Action Comics #1\" from June 1938. It shows a colorful illustration of Superman lifting a green car. There are people in the foreground reacting with surprise and fear. The cover has a bold red and yellow background.\n\nBelow this, there are additional single comic book covers displayed in a row from left to right:\n\n1. **The Warlord**: The cover features bold, vivid colors with an image of a muscular warrior wielding a sword against a dinosaur. The title is set against a dark background in an elaborate stylized font.\n\n2. **The Thing**: This cover features a group of heroes against a space-themed backdrop, with \"THE THING\" in a large red font at the top. The characters are dressed in futuristic outfits.\n\n3. **Star Wars**: The cover prominently displays the characters from the Star Wars universe, including Darth Vader, in front of starry space. The title \"STAR WARS\" is large, yellow, and bold.\n\nThe overall layout presents a historical context alongside visual examples of comic book covers, highlighting their cultural and collectible importance."
    },
    {
        "slide_number": 3,
        "text": [
            "My Comic Books",
            "I started collecting July 4th, 2021, and fell in love with the stories and artwork. I quickly learned certain artists and rarity of the covers could increase a book\u2019s value ten-fold in the first day."
        ],
        "description": "The image is a collage with a text portion on the left side and four comic book covers. Here's a detailed description:\n\n**Text:**\n- At the top left, there's a title that reads \"My Comic Books.\"\n- Below the title, a paragraph states: \"I started collecting July 4th, 2021, and fell in love with the stories and artwork. I quickly learned certain artists and rarity of the covers could increase a book\u2019s value ten-fold in the first day.\"\n\n**Comic Book Covers:**\n1. **Top Right Cover:**\n   - Titled \"We Don't Kill Spiders.\"\n   - Features an illustrated person holding a glowing red object with several dark shadows around. The background is blue with bright pink text.\n\n2. **Bottom Row - Left Cover:**\n   - Titled \"BRZRKR.\"\n   - Shows a monochrome figure in dark armor, holding a weapon, standing against a backdrop of large block letters in blue.\n\n3. **Bottom Row - Middle Cover:**\n   - Titled \"Stray Dogs: Dog Days.\"\n   - Has an image of a bloody envelope and a paw print, implying a mysterious or suspenseful theme, with mostly dark red and brown tones.\n\n4. **Bottom Row - Right Cover:**\n   - Titled \"Darth Vader.\"\n   - Depicts the iconic Star Wars character, Darth Vader, in full armor holding a lightsaber against a space-themed background with warm brown and gold colors.\n  \nThe comic book covers are arranged in two rows: one cover on the top right by itself, and three covers along the bottom. The text is adjacent to the bottom row. Each cover has its unique style and theme."
    },
    {
        "slide_number": 4,
        "text": [
            "My Comic Books",
            "As a Star Wars fan, the stories from the comics fill much of the gaps between the movies and TV shows. We get to see our beloved heroes, and villains, in their natural elements like we never see on the screen."
        ],
        "description": "The image is a slide titled \"My Comic Books,\" featuring a collection of Star Wars-themed comic book covers. \n\nAt the top, there's a heading in bold: \"My Comic Books.\" Below this, there's a paragraph of text: \"As a Star Wars fan, the stories from the comics fill much of the gaps between the movies and TV shows. We get to see our beloved heroes, and villains, in their natural elements like we never see on the screen.\"\n\nBeneath the text, four comic book covers are displayed. From left to right:\n\n1. **First Cover (Leftmost):** Features a character in a white outfit holding a red lightsaber. The background is dark, with the words \"Darth Vader\" prominently displayed at the bottom.\n\n2. **Second Cover:** Shows Darth Vader holding a red lightsaber, standing against a smoky background. \"Star Wars Darth Vader\" is written at the top in bold.\n\n3. **Third Cover:** This is a classic-style comic cover with multiple characters including Darth Vader in the center. The title \"Star Wars\" is written in bright yellow at the top.\n\n4. **Fourth Cover (Rightmost):** Features the character Chewbacca, a large, furry being with a serious expression. It is titled \"Han Solo & Chewbacca.\" This cover is positioned separately to the right of the other three covers.\n\nThe covers are aligned neatly, with the first three creating a row, and the fourth slightly offset to the right."
    },
    {
        "slide_number": 5,
        "text": [
            "My Collection",
            "The following are some stats from my collection"
        ],
        "description": "The image has a simple, minimalistic design. It consists of a plain white background with text aligned to the left side. The main heading, \"My Collection,\" is placed prominently in bold, black font. Below this heading, there is a subheading in smaller, gray font that reads, \"The following are some stats from my collection.\" There are no charts, graphs, or additional objects present in the image\u2014just these two lines of text centered towards the top left portion."
    },
    {
        "slide_number": 6,
        "text": [
            "Chart Title: ",
            "Series Name: Total",
            "Values: [835.0, 203.0, 116.0, 113.0, 80.0, 50.0, 21.0, 18.0, 17.0, 17.0, 13.0, 10.0, 8.0, 7.0, 7.0, 6.0, 5.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",
            "by publisher",
            "Contains more original stories (better in my opinion)",
            "This is so high due to Star Wars"
        ],
        "description": "The image is a bar chart with the title \"by publisher\" at the top left. The chart represents the total number of items attributed to various publishers along the x-axis, while the y-axis shows a numerical scale from 0 to 900.\n\n1. **X-Axis**: Represents different publishers. The list includes Marvel Comics at the far left, followed by DC Comics, Dark Horse Comics, and several others. Farther right, smaller publishers are also listed.\n\n2. **Y-Axis**: Ranges from 0 to 900, indicating the total count of items published.\n\n3. **Bars**: Each publisher has a vertical bar corresponding to the number of items. Notably:\n   - **Marvel Comics**: Has the tallest bar, reaching above 800, with a note stating, \"This is so high due to Star Wars.\"\n   - **DC Comics**: The second-highest bar, slightly above 200.\n   - **Dark Horse Comics**: Slightly above 100.\n   - The other publishers have significantly shorter bars, many below 100.\n\n4. **Annotations**:\n   - Next to the Marvel Comics bar, there is a text pointing at the height of the bar, explaining its height due to Star Wars.\n   - Above the middle publishers\u2019 shorter bars, there is a note stating, \"Contains more original stories (better in my opinion).\"\n\n5. **Overall Trend**: There is a steep drop-off in height after Marvel Comics and DC Comics, with minor fluctuations among the remaining publishers.\n\nThe visual presentation emphasizes the dominance of Marvel Comics in terms of volume, with DC Comics being a distant second, highlighting the contribution of Star Wars to Marvel\u2019s count."
    },
    {
        "slide_number": 7,
        "text": [
            "Series Name: Count of Series Name",
            "Values: [1.0, 10.0, 11.0, 9.0, 25.0, 15.0, 8.0, 2.0, 3.0, 2.0, 1.0, 6.0, 13.0, 14.0, 8.0, 14.0, 8.0, 2.0, 7.0, 6.0, 4.0, 3.0, 5.0, 1.0, 6.0, 2.0, 2.0, 2.0, 5.0, 12.0, 1.0, 10.0, 8.0, 27.0, 27.0, 11.0, 35.0, 21.0, 71.0, 369.0, 497.0, 298.0, 198.0]",
            "release years",
            "Started Collecting",
            "That was an expensive year"
        ],
        "description": "The image is a line graph titled \"release years.\" The x-axis represents years ranging from 1969 to 2024, marked in increments of a few years, while the y-axis represents an unspecified numerical value ranging from 0 to 600.\n\nThe data line starts low in 1969, with values remaining under 100 through to about 2014. There are minor fluctuations throughout these years. From around 2014, the line shows a steep upward trend, peaking sharply at around the year 2020, where it hits just below 500. After this peak, the line quickly descends, showing a sharp decline.\n\nTwo labels are present on the graph. One is at the peak, indicating \"That was an expensive year,\" suggesting that 2020 was notable. Another is near the start of the upward trend, marked \"Started Collecting,\" which points around the year 2014.\n\nThese labels provide context for the significant increase in numbers during this period, indicating critical points of interest in the timeline of data collection or events."
    },
    {
        "slide_number": 8,
        "text": [
            "Chart Title: Top Publishers",
            "Series Name: Read",
            "Values: [550.0, 100.0, 57.0, 43.0, 41.0]",
            "Series Name: Unread",
            "Values: [247.0, 87.0, 51.0, 64.0, 34.0]",
            "reading status",
            "Chart Title: All Comic Books",
            "Series Name: Total",
            "Values: [964.0, 816.0]"
        ],
        "description": "The image consists of two main parts: a pie chart on the left and a bar chart on the right. The overall topic is \"reading status\" of comic books.\n\n**Pie Chart (Left Side):**\n\n- Title: \"All Comic Books.\"\n- The pie chart is divided into two sections.\n- The larger section is labeled \"Read\" and is represented in dark blue, making up 54% of the chart.\n- The smaller section, labeled \"Unread,\" is in orange and accounts for 46% of the chart.\n- The chart visually shows that more comic books have been read than are unread.\n\n**Bar Chart (Right Side):**\n\n- Title: \"Top Publishers.\"\n- There are five publishers along the horizontal axis, each with its own bar rising upward:\n  1. Marvel Comics\n  2. DC Comics\n  3. BOOM! Studios\n  4. Dark Horse Comics\n  5. IDW Publishing\n- The height of the bars corresponds to the number of comic books, with specific amounts divided into read and unread.\n- The bars have two sections, similar in color to the pie chart: dark blue for read and orange for unread.\n- Marvel Comics' bar is the tallest, showing the highest number of comics, with a significant portion unread.\n- The other publishers have smaller bars, with varying ratios of read to unread comics.\n\nThis combination of charts provides an overview of comic book reading status, both generally and by specific publishers."
    },
    {
        "slide_number": 9,
        "text": [
            "Series Name: Star Wars Books",
            "Values: [554.0, 34.0, 67.0, 2.0]",
            "Series Name: Other Books",
            "Values: [243.0, 41.0, 40.0, 0.0]",
            "Series Name: ",
            "Values: [657.0, 324.0]",
            "star wars comics",
            "Star Wars\n67%",
            "All other books\n33%",
            "2"
        ],
        "description": "The image contains two main elements: a pie chart on the left and a bar graph on the right, with logos of comic publishers below the bar graph.\n\n### Pie Chart (Left Side)\n- **Title:** The chart is titled \"Star Wars Comics.\"\n- **Composition:** The pie chart is divided into two segments.\n  - **Larger Segment:** This takes up 67% of the circle, colored in blue, and labeled as \"Star Wars 67%.\"\n  - **Smaller Segment:** This occupies 33% of the circle, colored in orange, and labeled as \"All other books 33%.\"\n\n### Bar Graph (Right Side)\n- **Axes:** The vertical axis represents quantity, with numerical labels from 0 to 900 at intervals of 100. The horizontal axis lacks numerical labels but is associated with publisher logos.\n- **Bars:**\n  - The largest bar corresponds to the \"Marvel Comics\" logo. It is composed of a large blue lower section and a smaller orange upper section.\n  - The next two bars are significantly smaller and correspond to \"IDW Publishing\" and \"Dark Horse Comics\" logos. Each has a small orange segment on top of a blue base.\n  - There is also the \"Viz\" logo on the far right with a barely noticeable or absent bar.\n\n### Publisher Logos (Below the Graph)\n- The logos of four different comic publishers are displayed underneath the bar graph, aligned with the bars they represent:\n  1. **Marvel Comics:** Identified with the largest bar.\n  2. **IDW Publishing:** With a smaller bar.\n  3. **Dark Horse Comics:** Also with a smaller bar.\n  4. **Viz:** Associated with a negligible bar presence.\n\nThe image presents a visual comparison and distribution of \"Star Wars\" comics versus other comic books, showcasing dominance by Star Wars comics in both charts and identifying the publishers involved in this context."
    }
]

## Set up

Before we go further, let's setup our environment. Copy the `.env.example` to `.env` and add your OpenAI API key. Then run the following:

In [ ]:
%pip install openai python-dotenv

from dotenv import load_dotenv
load_dotenv()

Here's a basic function for calling GPT, sending in images and the user's question:

In [19]:
from openai import OpenAI
import base64
client = OpenAI()

total_completion = 0
total_prompt = 0

sys_prompt = """You are a helpful GPT. You will answer the user's question by using the provided images of slides from a PowerPoint deck.
Answer the user's question from the relevant images. Pay particular attention to the relationships of data and colors, especially charts and graphs.

Question:
"""

def call_gpt(text, images):
    global total_completion, total_prompt
    content = [
        {"type": "text", "text": f"{sys_prompt}{text}"},
    ]
    
    for image in images:
        with open(image, "rb") as image_file:
            image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
            content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image_base64}"
                }
            })

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": content
            }
        ],
        max_tokens=4000,
    )

    total_completion += response.usage.completion_tokens
    total_prompt += response.usage.prompt_tokens
    return response.choices[0].message.content

Using the same questions from before.

In [22]:
questions = [
    {
        "question": "How many Marvel books are there?",
        "keywords": ["Marvel"]
    },
    {
        "question": "How many Marvel books are Star Wars related?",
        "keywords": ["Marvel", "Star Wars"]
    },
    {
        "question": "What year did the collection start?",
        "keywords": ["start", "collect", "journey", "Collecting"]
    },
    {
        "question": "What are the top brands?",
        "keywords": ["brands", "publishers"]
    }
]

## The G in multi-modal RAG

Let's generate some answers using images, not text! The following includes the question, the images used, and then GPT's answer.

In [24]:
from IPython.display import Markdown, display


for question in questions:
    slides = [
        f"./comics/Slide{slide['slide_number']}.jpeg"
        for slide in all_data
        if any(k.lower() in slide["description"].lower() or k.lower() in ' '.join(slide['text']).lower() for k in question["keywords"])
    ]
    answer = call_gpt(question["question"], slides)

    display(Markdown(f"### {question['question']}"))
    for slide in slides:
        display(Markdown(f"<img src='{slide}' style='width:30%;' />"))
    display(Markdown(f"{answer}"))

print("DONE!")
print(f"Total {total_prompt} input tokens, {total_completion} output tokens")

### How many Marvel books are there?
<div style='display:flex;flex-wrap:wrap;'>

<img src='./comics/Slide6.jpeg' style='width:30%;' />

<img src='./comics/Slide8.jpeg' style='width:30%;' />

<img src='./comics/Slide9.jpeg' style='width:30%;' />

</div> The total number of Marvel books is approximately 750, as indicated by the bar under Marvel Comics in the top publishers chart on the second slide.

...

<img src='./comics/Slide6.jpeg' style='width:30%;' />

<img src='./comics/Slide8.jpeg' style='width:30%;' />

<img src='./comics/Slide9.jpeg' style='width:30%;' />

</div> The top brands, according to the charts, are:

1. Marvel Comics
2. DC Comics
3. BOOM! Studios
4. Dark Horse Comics
5. IDW Publishing 

These publishers are highlighted as top performers in all presented graphs. Marvel Comics has a notable lead due to the impact of Star Wars comics.

DONE!
Total 31314 input tokens, 319 output tokens


<img src="../images/yeee hoooo.PNG" style="width: 50%" alt="Comic panel of Han Solo exclaiming Yeee Hooo!" />

Not bad! 

However, look at how it handled **How many Marvel books are Star Wars related?**:

>
>The pie chart titled "star wars comics" shows that 67% of the comics are Star Wars related. Assuming the total number under Marvel Comics in the graph is approximately 800 (as indicated in the "by publisher" slide), this means there are around 536 Star Wars related Marvel books.
>

Awesome.

## A picture is worth a thousand words

When we use multi-modal RAG (sending images and asking questions) the machine can better identify the key portions of those images and provide valuable answers. If we convert to text first, all the machine has to work off of is the text. That text, though generated by GPT, may not include all of the context that pertains to the user's query. Let's see if I can prove this.

Let's create some new questions which may not be as easy from a textual summary.

In [25]:
hard_questions = [
    {
        "question": "What are the top brands that have better stories?",
        "keywords": ["publisher", "original"]
    },
    {
        "question": "How many books are from 1975?",
        "keywords": ["1975", "release"]
    },
    {
        "question": "How many Marvel books are read?",
        "keywords": ["Marvel", "reading"]
    }
]

Another GPT function for chatting with text, instead of images.

In [26]:
def call_gpt_text(text, context):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "developer", "content": "Help the user by answering their question from the provided context."},
            {
                "role": "user",
                "content": f"Context:\n{context}\n\nQuestion:\n{text}"
            }
        ],
        max_tokens=4000
    )
    return completion.choices[0].message.content

Now let's loop through the questions, and make two GPT calls, one for the vision description, and one using the images.

In [27]:
total_prompt = 0
total_completion = 0

for question in hard_questions:
    slides = [
        slide
        for slide in all_data
        if any(k.lower() in slide["description"].lower() or k.lower() in ' '.join(slide['text']).lower() for k in question["keywords"])
    ]
    images = [f"./comics/Slide{slide['slide_number']}.jpeg" for slide in slides]
    image_answer = call_gpt(question["question"], images)
    text_answer = call_gpt_text(question["question"], '"\n*****\n"'.join([slide["description"] for slide in slides]))

    display(Markdown(f"### {question['question']}"))
    for slide in slides:
        src = f"./comics/Slide{slide['slide_number']}.jpeg"
        display(Markdown(f"<img src='{src}' style='width:30%;' />"))
    display(Markdown(f"""### Multi-modal Answer
{image_answer}

### Vision Description Answer
{text_answer}

"""))

print("DONE!")
print(f"Total {total_prompt} input tokens, {total_completion} output tokens")

### What are the top brands that have better stories?

<img src='./comics/Slide6.jpeg' style='width:30%;' />

<img src='./comics/Slide8.jpeg' style='width:30%;' />

<img src='./comics/Slide9.jpeg' style='width:30%;' />

### Multi-modal Answer
The top brands that contain more original stories, according to the first slide, are Image Comics, with IDW Publishing and Dark Horse Comics contributing significantly due to their Star Wars content. Image Comics is highlighted as containing more original stories "in my opinion."

### Vision Description Answer
According to the information provided, the top brands with better stories are associated with the middle publishers in the bar chart under the section labeled "by publisher." A note mentions that these publishers, which include those with bars slightly above and below 100, contain more original stories and are considered better in someone's opinion. Specifically, Dark Horse Comics and other smaller publishers with bars below 100 are highlighted as having better, more original stories.



...

<img src='./comics/Slide6.jpeg' style='width:30%;' />

<img src='./comics/Slide8.jpeg' style='width:30%;' />

<img src='./comics/Slide9.jpeg' style='width:30%;' />

### Multi-modal Answer
To determine how many Marvel books are read, we can analyze the data from the slides:

1. **Total Marvel Books**: The bar chart from the first slide shows that Marvel has the highest number of books among the publishers, close to 800.

2. **Reading Status for Marvel**: On the second slide, the top publishers' bar chart shows the read (blue) and unread (orange) portions for Marvel. It appears that the read portion is slightly more than half.

3. **Calculation**: If Marvel has about 800 books in total and more than half are read, then approximately 400 to 450 Marvel books are read.

Hence, around 400-450 Marvel books are read.

### Vision Description Answer
The context provided does not specify the exact number of Marvel books that are read. However, in the bar chart titled "Top Publishers," the Marvel Comics bar is the tallest and divided into two sections: dark blue for read and orange for unread. To find the number of read Marvel books, one would need to determine the proportion of the dark blue section in the Marvel Comics bar relative to the total height of that bar. Unfortunately, without specific numerical values or additional data from the chart, the precise number cannot be calculated from the given information.



DONE!
Total 5303 input tokens, 208 output tokens


## Multi-modal RAG is better

Let's check out the questions and who gave a better answer.

**What are the top brands that have better stories?** Multi-modal is better, although the vision description answer feels more thoughtful...

**How many books are from 1975?** Multi-modal killed it. Vision description has no idea.

**How many Marvel books are read?** Multi-modal has some good reasoning, but it was the wrong answer. Vision description has no idea.

Is multi-modal better?

<img src="../images/yes it is.jpg" style="width: 50%" alt="Comic panel fo Darth Vader saying Yes It Is" />

Multi-modal is clearly a better experience, although not perfect (nothing is perfect).


## Implementing multi-modal RAG

As we walked through here, it's not simply giving the model images, but the first important piece is finding the right images to give the LLM. You may still want to run vision on ingestion, as we discussed in the [other notebook](./powerpoint.ipynb), so you have a better search experience, and then send those slides into the LLM.

Keep in mind, my example here is one PowerPoint file. If this was against a full corpus of knowledge, that could be millions of slides. I suggest including more context for each slide, so the machine can understand what the slide is above, in the context of the deck it's in. 

And yes, there's a cost with all of this. As I shared in the last notebook, I'll say it again here:

>In reality, most of us live within budgets, have end-user performance expectations, speed to market requirements, blah blah blah. It'll come down to the business stakeholders to see the value in using vision, compared to the current value you're delivering, and decide the cost is worth it. 

